In [13]:
import numpy as np 
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications import ResNet50
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
import keras
from scipy.io import loadmat
import h5py
from PIL import Image
import os, zipfile
import numpy as np
from skimage.feature import greycomatrix, greycoprops
from skimage import io, color, img_as_ubyte
import random

init_notebook_mode(connected=True)
RANDOM_SEED = 123

In [14]:
import uuid

In [15]:
ims = './matlab'
directory_to_extract_to = './matlab/'

In [16]:
X = []
y = []

In [17]:
# import zipfile
# for file in os.listdir(ims):
#     if not file.startswith('.'):
#         with zipfile.ZipFile(os.path.join("matlab/",file), 'r') as zip_ref:
#             zip_ref.extractall(directory_to_extract_to)

In [18]:
ones = './train_mat/1'
twos = './train_mat/2'
threes = './train_mat/3'

In [19]:
for count,val in enumerate(os.listdir(ims)):
    if not val.startswith('.'):
        file = os.path.join(ims,val)
        f = h5py.File(file,'r')
        data = f.get('cjdata/image')
        label = f.get('cjdata/label')
        data = np.array(data)
        if label[0][0]==1.0:

            cv2.imwrite(os.path.join(ones,str(count)+'.jpg'),data)
            X.append(data)
            y.append(label[0][0])
        elif label[0][0]==2.0:
            cv2.imwrite(os.path.join(twos,str(count)+'.jpg'),data)
            X.append(data)
            y.append(label[0][0])
        elif label[0][0]==3.0:
            cv2.imwrite(os.path.join(threes,str(count)+'.jpg'),data)
            X.append(data)
            y.append(label[0][0])




In [20]:
directory = './train_mat/'

In [21]:
shufflingz = os.listdir(directory)

In [22]:
random.shuffle(shufflingz)


In [24]:
for val in os.listdir(directory):
    if not val.startswith('.'):
        length = len(os.listdir(directory + val))
        for (x, y) in enumerate(os.listdir(directory + val)):
            img = directory + val + '/' + y
            if x % 30 ==0:
                shutil.copy(img, 'test_new/' + val + '/' + y)
            elif x % 10 == 0:
                shutil.copy(img, 'val_new/'+ val + '/' + y)
            else:
                shutil.copy(img, 'train_new/'+ val + '/' + y)

In [ ]:
# for val in os.listdir(directory):
#     if not val.startswith('.'):
#         length = len(os.listdir(directory + val))
#         for (x, y) in enumerate(os.listdir(directory + val)):
#             img = directory + val + '/' + y
#             if x % 30:
#                 shutil.copy(img, 'test_new/' + val + '/' + y)
#             elif x % 0.7*length:
#                 shutil.copy(img, 'train_new/'+ val + '/' + y)
#             else:
#                 shutil.copy(img, 'val_new/'+ val + '/' + y)

In [10]:
def crops(type_of_data):
    set_new = []

    for img in type_of_data:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.erode(thresh, None, iterations=2)
        thresh = cv2.dilate(thresh, None, iterations=2)
        thresh = cv2.convertScaleAbs(thresh)
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        c = max(cnts, key=cv2.contourArea)

        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        ADD_PIXELS = 0
        new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
        set_new.append(new_img)
    return np.array(set_new)


In [11]:
def loading_data(training_directory):
    counter = 0
    storage = dict()
    X = []
    y = []
    for val in sorted(os.listdir(training_directory)):
        if not val.startswith('.'):
            storage[counter] = val
            for valtwo in os.listdir(training_directory + val):
                if not valtwo.startswith('.'):
                    img = cv2.imread(training_directory+val+'/'+valtwo)
                    X.append(img)
                    y.append(counter)
            counter+=1
    X = np.array(X)
    y = np.array(y)
    
    return X,y,storage
    

In [12]:
training_directory = './train_new/'
testing_directory = './test_new/'
validation_directory = './val_new/'

In [13]:
X_train,y_train,storage = loading_data(training_directory)
X_test, y_test, _ = loading_data(testing_directory)
X_val, y_val,_ = loading_data(validation_directory)

In [14]:
X_train_crop = crops(X_train)
X_val_crop = crops(X_val)
X_test_crop = crops(X_test)

In [15]:
def save_crops(Xs,Ys, folder):
    val = -1

    for (image, image_class) in zip(Xs,Ys):
        val+=1
        if image_class == 0:
            cv2.imwrite(os.path.join(folder,'1/'+str(val)+'.jpg'),image)
        elif image_class == 1:
            cv2.imwrite(os.path.join(folder,'2/'+str(val)+'.jpg'),image)
        else:
            cv2.imwrite(os.path.join(folder,'3/'+str(val)+'.jpg'),image)

            

In [16]:
save_crops(X_train_crop, y_train, folder = 'train_crop_new/')


In [17]:
save_crops(X_test_crop, y_test, folder = 'test_crop_new/')


In [18]:
save_crops(X_val_crop, y_val, folder = 'val_crop_new/')

In [19]:
len(os.listdir('./train_crop_new/1'))

468

In [60]:
IMG_SIZE = (224,224)

In [2]:
def resize_ims(type_of_data,size):
    new_set = []
    
    for val in type_of_data:
        val = cv2.resize(val, dsize=size,interpolation=cv2.INTER_CUBIC)
        
        new_set.append(preprocess_input(val))
    return np.array(new_set)

In [207]:
X_train_prep = resize_ims(X_train_crop,size=IMG_SIZE)
X_test_prep = resize_ims(X_test_crop,size=IMG_SIZE)
X_val_prep = resize_ims(X_val_crop,size=IMG_SIZE)

In [5]:
TRAIN_DIR = 'train_crop_new/'
VAL_DIR = 'val_crop_new'


In [9]:
cd train_crop_new


/Users/shonbutani/Downloads/Brain_Tumor_Dataset2/train_crop_new


In [10]:
cd 1

/Users/shonbutani/Downloads/Brain_Tumor_Dataset2/train_crop_new/1
